In [12]:
from dota2_api import get_api_json

match_id = 3159781877
match_info = get_api_json('https://api.opendota.com/api/matches/{}'.format(match_id))
match_info['objectives']

[{'key': 7,
  'player_slot': 4,
  'slot': 4,
  'time': 307,
  'type': 'CHAT_MESSAGE_FIRSTBLOOD'},
 {'player_slot': 128,
  'slot': 5,
  'team': 3,
  'time': 737,
  'type': 'CHAT_MESSAGE_TOWER_KILL'},
 {'player_slot': 128,
  'slot': 5,
  'team': 3,
  'time': 801,
  'type': 'CHAT_MESSAGE_TOWER_KILL'},
 {'team': 2, 'time': 832, 'type': 'CHAT_MESSAGE_ROSHAN_KILL'},
 {'player_slot': 4, 'slot': 4, 'time': 833, 'type': 'CHAT_MESSAGE_AEGIS'},
 {'slot': -1, 'team': 2, 'time': 914, 'type': 'CHAT_MESSAGE_TOWER_KILL'},
 {'player_slot': 4,
  'slot': 4,
  'team': 2,
  'time': 995,
  'type': 'CHAT_MESSAGE_TOWER_KILL'},
 {'player_slot': 3,
  'slot': 3,
  'team': 2,
  'time': 1060,
  'type': 'CHAT_MESSAGE_TOWER_KILL'},
 {'player_slot': 130,
  'slot': 7,
  'team': 3,
  'time': 1148,
  'type': 'CHAT_MESSAGE_TOWER_KILL'},
 {'player_slot': 128,
  'slot': 5,
  'team': 3,
  'time': 1269,
  'type': 'CHAT_MESSAGE_TOWER_KILL'},
 {'team': 3, 'time': 1388, 'type': 'CHAT_MESSAGE_ROSHAN_KILL'},
 {'player_slot': 129,

In [21]:
def is_destroy_first_tower_induce_win(match_info):
    if 'radiant_win' not in match_info:
        return None
    win_camp = 'Radiant' if match_info['radiant_win'] is True else 'Dire'
    if match_info['objectives']:
        for objective in sorted(match_info['objectives'], key=lambda x:x['time']):
            if 'key' in objective and 'tower1' in str(objective['key']):
                tower_camp = 'Dire' if 'goodguys' in objective['key'] else 'Radiant'
                if win_camp == tower_camp:
                    return True
                else:
                    return False
    return None

def is_destroy_mid_tower_induce_win(match_info):
    if 'radiant_win' not in match_info:
        return None
    win_camp = 'Radiant' if match_info['radiant_win'] is True else 'Dire'
    if match_info['objectives']:
        for objective in sorted(match_info['objectives'], key=lambda x:x['time']):
            if 'key' in objective and 'tower1_mid' in str(objective['key']):
                tower_camp = 'Dire' if 'goodguys' in objective['key'] else 'Radiant'
                if win_camp == tower_camp:
                    return True
                else:
                    return False
    return None

def is_kill_roshan_induce_win(match_info):
    if 'radiant_win' not in match_info:
        return None
    win_camp = 'Radiant' if match_info['radiant_win'] is True else 'Dire'
    if match_info['objectives']:
        for objective in sorted(match_info['objectives'], key=lambda x:x['time']):
            if 'type' in objective and 'CHAT_MESSAGE_AEGIS' in str(objective['type']):
                roshan_camp = 'Radiant' if objective['slot'] <= 4 else 'Dire'
                if win_camp == roshan_camp:
                    return True
                else:
                    return False
    return None

def is_firts_blood_induce_win(match_info):
    if 'radiant_win' not in match_info:
        return None
    win_camp = 'Radiant' if match_info['radiant_win'] is True else 'Dire'
    if match_info['objectives']:
        for objective in sorted(match_info['objectives'], key=lambda x:x['time']):
            if 'type' in objective and 'CHAT_MESSAGE_FIRSTBLOOD' in str(objective['type']):
                fb_camp = 'Radiant' if objective['slot'] <= 4 else 'Dire'
                if win_camp == fb_camp:
                    return True
                else:
                    return False
    return None

In [26]:
from dota2_api import get_api_json

sql = """SELECT match_id
FROM match_patch where patch >= '7' 
ORDER BY RANDOM()
LIMIT 10000"""

# sql = """SELECT
# matches.match_id
# FROM matches
# JOIN match_patch using(match_id)
# JOIN leagues using(leagueid)
# where patch >= '7' 
# ORDER BY RANDOM()
# LIMIT 100
# """

match_ids = get_api_json('https://api.opendota.com/api/explorer?sql={}'.format(sql))['rows']

In [27]:
match_count = 0
mid_tower_induce_win_count = 0
kill_roshan_induce_win_count = 0
first_tower_induce_win_count = 0
firts_blood_induce_win_count = 0

for mi in match_ids:
    match_info = get_api_json('https://api.opendota.com/api/matches/{}'.format(mi['match_id']))
    destroy_mid_tower_induce_win = is_destroy_mid_tower_induce_win(match_info)
    kill_roshan_induce_win = is_kill_roshan_induce_win(match_info)
    destroy_first_tower_induce_win = is_destroy_first_tower_induce_win(match_info)
    firts_blood_induce_win = is_firts_blood_induce_win(match_info)
    if destroy_first_tower_induce_win is not None and kill_roshan_induce_win is not None \
    and destroy_first_tower_induce_win is not None and firts_blood_induce_win is not None:
        match_count += 1
        if destroy_mid_tower_induce_win is True:
            mid_tower_induce_win_count += 1
        if kill_roshan_induce_win is True:
            kill_roshan_induce_win_count += 1
        if destroy_first_tower_induce_win is True:
            first_tower_induce_win_count += 1
        if firts_blood_induce_win is True:
            firts_blood_induce_win_count += 1

print(match_count, mid_tower_induce_win_count, kill_roshan_induce_win_count, first_tower_induce_win_count, firts_blood_induce_win_count)

1979 1280 1493 1218 1086


In [8]:
from math import sqrt

def get_confidence_intervals(count, true_count):
    p = float(true_count) / count
    se = sqrt(p*(1-p)/count)
    return (round(p - 1.96 * se, 3), round(p + 1.96 * se, 3) )

print(get_confidence_intervals(107 ,67))

(0.534, 0.718)


In [ ]:
# sql
# SELECT count(*)
# FROM match_patch where patch >= '7' 

# SELECT match_id
# FROM match_patch where patch >= '7' 
# ORDER BY RANDOM()
# LIMIT 100